In [12]:
import os
import json
from bokeh.io import output_notebook, output_file, show
from bokeh.models import HoverTool, Paragraph, Div, ColumnDataSource
from bokeh.plotting import figure
from bokeh.layouts import layout

#Search the directory for all Jupyter notebook files and add the cells of each file to an array
et_dir="autograded/user001/entrance_test"
cells_all=[]
for file in os.listdir(et_dir):
    if file.endswith(".ipynb"):
        with open(os.path.join(et_dir, file), mode= "r", encoding= "utf-8") as f:
            myfile = json.loads(f.read())
            cells_all.extend(myfile['cells'])

#Define the standard tags to compare against
tags_standard = ['visualization', 'functionals','sequences', 'conditionals', 'loops', 'operators', 'data']

#Initialize cell counts and totals
counts=[0,0,0,0,0,0,0]
totals = [0,0,0,0,0,0,0]

#For each cell, match the tags against the standard tags and update the cell counts and totals
for cell in cells_all:
    if 'tags' in cell['metadata']:
        tags = cell['metadata']['tags']
        for tag in tags:
            for index in range(len(tags_standard)):
                if tag == tags_standard[index]:
                    totals[index]= totals[index]+1
                    if len(cell['outputs']) == 0:
                        counts[index]= counts[index]+1
                    break

# print(counts)
# print(totals)

#Set the details of the output file
# output_notebook()
output_file('ET_dashboard.html', title="Entrance Test dashboard")

#Define the concepts, modules and descriptions
concepts = ['Visualization', 'Functionals', 'Sequences', 'Conditionals', 'Loops', 'Operators', 'Data']
modules = ['Basic plotting', 'Code organization', 'Code organization', 'Control flow', 'Control flow', 'Variables', 'Variables']

# counts = [2, 3, 4, 8, 8, 9, 8]
# totals = [10,10,10,10,10,10,8]
y_axis = []
colors=[]

#Compute the y-axis and colors for bars
for i in range(len(counts)):
    y_axis.append((counts[i]/totals[i])*10)
    if y_axis[i] < 8:
        colors.append("indianred")
    else:
        colors.append("steelblue")

description = ['Visualization involves viewing data using graphs or charts',
               'Functionals are blocks that a program is divided into',
               'Sequences are  an activity/task expressed as a series of individual steps',
               'Conditionals allow making logical decisions based on a condition',
               'Loops allow repetition of a statement multiple times based on a condition',
               'Operators provide support for mathematical, logical, and string expressions',
               'Data involves storing, retrieving, and updating values']



#Create the columnDataSource for the graph        
source = ColumnDataSource(data=dict(concepts=concepts, counts=counts, colors=colors, totals=totals, desc=description, module = modules, y_axis=y_axis))

#Define the hover tooltip
hover = HoverTool(
        tooltips=[
            ("Score", "@y_axis"),
            ("Passed tests", "@counts/@totals"),
            ("Description", "@desc"),
            ("Module", "@module")
        ]
    )

#Define the header div
div_header = Div(text="""<h1>\tModule: Entrance Test</h1>""", background="steelblue", width=1100, height=50, style={'color': 'white', 'padding-left':'30px'})

#Define the main graph
p = figure(y_range=concepts, x_range=(0,10), title="Computational Thinking concepts scores", toolbar_location=None, tools=[hover], width=700)
p.hbar(y='concepts', height=0.5, left=0,
       right='y_axis', color='colors', source=source)
p.y_range.range_padding = 0.1
p.ygrid.grid_line_color = None
p.xgrid.grid_line_color = None
p.axis.major_label_text_font_size = "15pt"
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.title.text_font_size = "25px"
p.title.align = "center"
p.xaxis.axis_label = "Score out of 10"
# show(p)

#Define the div with description of each concept
text_div = """</br></br></br>
              <h3><b>Data</b> involves storing, retrieving, and updating values. <i>Module: Variables</i></br></br>
              <b>Operators</b> provide support for mathematical, logical, and string expressions. <i>Module: Variables</i></br></br>
              <b>Loops</b> allow repetition of a statement multiple times based on a condition. <i>Module: Control Flow</i></br></br>
              <b>Conditionals</b> allow making logical decisions based on a condition. <i>Module: Control Flow</i></br></br>
              <b>Sequences</b> are  an activity/task expressed as a series of individual steps. <i>Module: Code Organization</i></br></br>
              <b>Functionals</b> are blocks that a program is divided into. <i>Module: Code Organization</i></br></br>
              <b>Visualization</b> involves viewing data using graphs or charts. <i>Module: Basic Plotting</i></br></h3>"""
div_concepts = Div(text=text_div, width=400, style={'padding-left':'20px'})

#Define the action div
div_action = Div(text="""<h2><i>ACTION</i>: Start the learning materials at the first module that has a red progress bar</h2>""", width=900, height=50)

#Set the dashboard layout and display it
l = layout([
    [div_header],
    [p, div_concepts],
    [div_action]
])
show(l)     